# 05 - Encapsulamiento y Propiedades

## ¿Qué es el Encapsulamiento?

El encapsulamiento es el principio de ocultar los detalles internos de una clase y controlar el acceso a sus atributos y métodos.

### Analogía del mundo real

Piensa en un cajero automático:
- **Público**: Puedes ver la pantalla, insertar tu tarjeta, ingresar tu PIN
- **Privado**: No puedes acceder al sistema interno, manipular directamente el dinero o modificar el software

El cajero expone solo lo necesario y protege lo crítico.

---

## Niveles de Acceso en Python

Python usa convenciones de nomenclatura para indicar el nivel de acceso:

| Tipo | Sintaxis | Descripción | Ejemplo |
|------|----------|-------------|----------|
| Público | `nombre` | Accesible desde cualquier lugar | `self.nombre` |
| Protegido | `_nombre` | Por convención, uso interno (no forzado) | `self._edad` |
| Privado | `__nombre` | Name mangling, más difícil de acceder | `self.__saldo` |

---

## Atributos Públicos, Protegidos y Privados

In [ ]:
class Persona:
    def __init__(self, nombre, edad, saldo):
        self.nombre = nombre          # Público
        self._edad = edad             # Protegido (por convención)
        self.__saldo = saldo          # Privado
    
    def mostrar_info(self):
        print(f"Nombre: {self.nombre}")
        print(f"Edad: {self._edad}")
        print(f"Saldo: ${self.__saldo}")

persona = Persona("Ana", 25, 10000)

# Público - Acceso directo
print(f"Público: {persona.nombre}")

# Protegido - Técnicamente accesible, pero no deberías
print(f"Protegido: {persona._edad}")

# Privado - No accesible directamente
try:
    print(persona.__saldo)
except AttributeError as e:
    print(f"Error al acceder privado: {e}")

# Pero el método de la clase sí puede acceder
print("\nDesde el método:")
persona.mostrar_info()

### Name Mangling

Python internamente renombra los atributos privados para hacerlos más difíciles de acceder:

In [ ]:
# Técnicamente aún puedes acceder, pero no es recomendado
print(f"\nAccediendo con name mangling: {persona._Persona__saldo}")

# Ver todos los atributos del objeto
print("\nTodos los atributos:")
print([attr for attr in dir(persona) if not attr.startswith('__')])

---

## ¿Por qué Usar Encapsulamiento?

In [ ]:
# SIN ENCAPSULAMIENTO - Propenso a errores
class CuentaBancariaMala:
    def __init__(self, saldo):
        self.saldo = saldo  # Público

cuenta = CuentaBancariaMala(1000)

# Cualquiera puede modificar el saldo directamente
cuenta.saldo = -5000  # ¡No tiene sentido!
print(f"Saldo sin validación: ${cuenta.saldo}")

In [ ]:
# CON ENCAPSULAMIENTO - Controlado y seguro
class CuentaBancariaBuena:
    def __init__(self, saldo_inicial):
        self.__saldo = saldo_inicial  # Privado
    
    def depositar(self, cantidad):
        if cantidad > 0:
            self.__saldo += cantidad
            print(f"Depósito exitoso: ${cantidad}")
        else:
            print("Error: Cantidad debe ser positiva")
    
    def retirar(self, cantidad):
        if cantidad > 0:
            if cantidad <= self.__saldo:
                self.__saldo -= cantidad
                print(f"Retiro exitoso: ${cantidad}")
            else:
                print("Error: Fondos insuficientes")
        else:
            print("Error: Cantidad debe ser positiva")
    
    def consultar_saldo(self):
        return self.__saldo

cuenta = CuentaBancariaBuena(1000)
print(f"Saldo inicial: ${cuenta.consultar_saldo()}")

# Intentar hacer lo mismo que antes
# cuenta.__saldo = -5000  # No funciona

# Debe usar los métodos controlados
cuenta.retirar(2000)  # Validación: fondos insuficientes
cuenta.depositar(500)
print(f"Saldo final: ${cuenta.consultar_saldo()}")

---

## Getters y Setters Tradicionales

Los getters y setters son métodos para acceder y modificar atributos privados:

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.__nombre = nombre
        self.__edad = edad
    
    # Getter para nombre
    def get_nombre(self):
        return self.__nombre
    
    # Setter para nombre
    def set_nombre(self, nombre):
        if nombre and len(nombre.strip()) > 0:
            self.__nombre = nombre
        else:
            print("Error: Nombre inválido")
    
    # Getter para edad
    def get_edad(self):
        return self.__edad
    
    # Setter para edad con validación
    def set_edad(self, edad):
        if 0 <= edad <= 150:
            self.__edad = edad
        else:
            print("Error: Edad debe estar entre 0 y 150")

persona = Persona("Ana", 25)

# Usar getters
print(f"Nombre: {persona.get_nombre()}")
print(f"Edad: {persona.get_edad()}")

# Usar setters
persona.set_edad(30)
print(f"Nueva edad: {persona.get_edad()}")

# Validación en acción
persona.set_edad(200)  # Error

---

## Propiedades con @property (Forma Pythonica)

Python tiene una forma más elegante: el decorador `@property`

In [ ]:
class Persona:
    def __init__(self, nombre, edad):
        self.__nombre = nombre
        self.__edad = edad
    
    # Propiedad para nombre (getter)
    @property
    def nombre(self):
        return self.__nombre
    
    # Setter para nombre
    @nombre.setter
    def nombre(self, nombre):
        if nombre and len(nombre.strip()) > 0:
            self.__nombre = nombre
        else:
            raise ValueError("Nombre inválido")
    
    # Propiedad para edad (getter)
    @property
    def edad(self):
        return self.__edad
    
    # Setter para edad
    @edad.setter
    def edad(self, edad):
        if 0 <= edad <= 150:
            self.__edad = edad
        else:
            raise ValueError("Edad debe estar entre 0 y 150")

persona = Persona("Ana", 25)

# Ahora se usa como atributos normales
print(f"Nombre: {persona.nombre}")  # Sin paréntesis
print(f"Edad: {persona.edad}")

# Modificar (con validación)
persona.edad = 30
print(f"Nueva edad: {persona.edad}")

# Intentar valor inválido
try:
    persona.edad = 200
except ValueError as e:
    print(f"Error: {e}")

### Ventajas de @property

1. **Sintaxis limpia**: Se usa como atributo normal
2. **Validación automática**: El setter valida los datos
3. **Retrocompatibilidad**: Puedes convertir atributos públicos en propiedades sin romper código existente
4. **Computed properties**: Puedes calcular valores dinámicamente

---

## Propiedades de Solo Lectura

In [ ]:
class Rectangulo:
    def __init__(self, ancho, alto):
        self.__ancho = ancho
        self.__alto = alto
    
    @property
    def ancho(self):
        return self.__ancho
    
    @ancho.setter
    def ancho(self, ancho):
        if ancho > 0:
            self.__ancho = ancho
        else:
            raise ValueError("Ancho debe ser positivo")
    
    @property
    def alto(self):
        return self.__alto
    
    @alto.setter
    def alto(self, alto):
        if alto > 0:
            self.__alto = alto
        else:
            raise ValueError("Alto debe ser positivo")
    
    # Propiedad calculada (solo lectura)
    @property
    def area(self):
        return self.__ancho * self.__alto
    
    # Otra propiedad calculada
    @property
    def perimetro(self):
        return 2 * (self.__ancho + self.__alto)

rect = Rectangulo(5, 10)

print(f"Ancho: {rect.ancho}")
print(f"Alto: {rect.alto}")
print(f"Área: {rect.area}")  # Calculado automáticamente
print(f"Perímetro: {rect.perimetro}")

# Modificar dimensiones
rect.ancho = 7
print(f"\nNueva área: {rect.area}")  # Se recalcula automáticamente

# Intentar modificar área (error, no tiene setter)
try:
    rect.area = 100
except AttributeError as e:
    print(f"\nError: {e}")

---

## Ejemplo Completo: Clase Temperatura

In [ ]:
class Temperatura:
    def __init__(self, celsius=0):
        self.__celsius = celsius
    
    # Propiedad Celsius
    @property
    def celsius(self):
        return self.__celsius
    
    @celsius.setter
    def celsius(self, valor):
        if valor < -273.15:  # Cero absoluto
            raise ValueError("Temperatura no puede ser menor al cero absoluto (-273.15°C)")
        self.__celsius = valor
    
    # Propiedad Fahrenheit (calculada)
    @property
    def fahrenheit(self):
        return (self.__celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, valor):
        # Convertir y usar el setter de celsius (incluye validación)
        self.celsius = (valor - 32) * 5/9
    
    # Propiedad Kelvin (calculada)
    @property
    def kelvin(self):
        return self.__celsius + 273.15
    
    @kelvin.setter
    def kelvin(self, valor):
        if valor < 0:
            raise ValueError("Kelvin no puede ser negativo")
        self.celsius = valor - 273.15
    
    def info(self):
        print(f"Celsius: {self.celsius:.2f}°C")
        print(f"Fahrenheit: {self.fahrenheit:.2f}°F")
        print(f"Kelvin: {self.kelvin:.2f}K")

# Crear temperatura
temp = Temperatura(25)
print("Temperatura inicial:")
temp.info()

# Modificar en Fahrenheit
print("\nCambiando a 98.6°F:")
temp.fahrenheit = 98.6
temp.info()

# Modificar en Kelvin
print("\nCambiando a 300K:")
temp.kelvin = 300
temp.info()

# Intentar valor inválido
print("\nIntentando temperatura inválida:")
try:
    temp.celsius = -300
except ValueError as e:
    print(f"Error: {e}")

---

## Ejemplo: Sistema de Validación de Email

In [ ]:
import re

class Usuario:
    def __init__(self, nombre, email, edad):
        self.__nombre = nombre
        self.email = email  # Usa el setter con validación
        self.edad = edad    # Usa el setter con validación
    
    @property
    def nombre(self):
        return self.__nombre
    
    @nombre.setter
    def nombre(self, nombre):
        if nombre and len(nombre.strip()) >= 2:
            self.__nombre = nombre.strip()
        else:
            raise ValueError("Nombre debe tener al menos 2 caracteres")
    
    @property
    def email(self):
        return self.__email
    
    @email.setter
    def email(self, email):
        # Validación básica de email
        patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if re.match(patron, email):
            self.__email = email.lower()
        else:
            raise ValueError("Email inválido")
    
    @property
    def edad(self):
        return self.__edad
    
    @edad.setter
    def edad(self, edad):
        if 13 <= edad <= 120:
            self.__edad = edad
        else:
            raise ValueError("Edad debe estar entre 13 y 120")
    
    # Propiedad de solo lectura
    @property
    def es_mayor_edad(self):
        return self.__edad >= 18
    
    @property
    def dominio_email(self):
        return self.__email.split('@')[1]
    
    def info(self):
        print(f"Nombre: {self.nombre}")
        print(f"Email: {self.email}")
        print(f"Dominio: {self.dominio_email}")
        print(f"Edad: {self.edad}")
        print(f"Mayor de edad: {'Sí' if self.es_mayor_edad else 'No'}")

# Crear usuario válido
usuario = Usuario("Ana García", "ana.garcia@example.com", 25)
usuario.info()

print("\n" + "-" * 40)

# Probar validaciones
print("\nProbando validaciones:")
try:
    usuario.email = "email_invalido"
except ValueError as e:
    print(f"Error email: {e}")

try:
    usuario.edad = 10
except ValueError as e:
    print(f"Error edad: {e}")

---

## Ejercicios Prácticos

### Ejercicio 1: Clase CuentaBancaria Completa

Crea una clase `CuentaBancaria` con:
- Atributos privados: titular, saldo, numero_cuenta
- Propiedades con validación para titular (no vacío) y saldo (no negativo)
- Propiedad de solo lectura para numero_cuenta
- Métodos: depositar, retirar
- Propiedad calculada: saldo_formateado (retorna "$X,XXX.XX")

In [ ]:
# Tu código aquí


### Ejercicio 2: Clase Producto con Descuento

Crea una clase `Producto` con:
- Atributos privados: nombre, precio, descuento_porcentaje (0-100)
- Propiedades con validación para todos los atributos
- Propiedad calculada: precio_final (precio - descuento)
- Propiedad calculada: ahorro (cuánto se ahorra con el descuento)
- Método: info() que muestra toda la información

In [ ]:
# Tu código aquí


### Ejercicio 3: Clase Estudiante con Promedio

Crea una clase `Estudiante` con:
- Atributos privados: nombre, matricula, calificaciones (lista)
- Propiedades con getters y setters para nombre y matrícula
- Método: agregar_calificacion(calificacion) con validación (0-10)
- Propiedad calculada de solo lectura: promedio
- Propiedad calculada de solo lectura: estado ("Aprobado" si promedio >= 6, "Reprobado" si < 6)
- Propiedad de solo lectura: total_materias (número de calificaciones)

In [ ]:
# Tu código aquí


### Ejercicio 4: Sistema de Gestión de Contraseñas (Desafío)

Crea una clase `Usuario` con:

1. Atributos privados:
   - username (solo lectura después de crear)
   - __password_hash (nunca exponer directamente)
   - email
   - fecha_creacion (solo lectura)
   - intentos_fallidos (privado)

2. Propiedades:
   - username (solo getter)
   - email (getter y setter con validación de formato)
   - fecha_creacion (solo getter)
   - cuenta_bloqueada (solo getter, True si intentos_fallidos >= 3)

3. Métodos:
   - cambiar_password(password_actual, password_nueva): Valida la actual y cambia
   - verificar_password(password): Retorna True/False, incrementa intentos_fallidos
   - desbloquear_cuenta(): Resetea intentos_fallidos (solo si conoces la password)
   - validar_fortaleza_password(password): Retorna True si cumple:
     - Al menos 8 caracteres
     - Al menos una mayúscula
     - Al menos una minúscula
     - Al menos un número

Simula hash de password con: `hash(password)` (no usar en producción real)

In [ ]:
# Tu código aquí


---

## Resumen

En este cuaderno aprendiste:

- ✅ Qué es el encapsulamiento y por qué es importante
- ✅ Niveles de acceso: público, protegido, privado
- ✅ Getters y setters tradicionales
- ✅ El decorador `@property` y su sintaxis
- ✅ Propiedades de solo lectura
- ✅ Propiedades calculadas dinámicamente
- ✅ Validación de datos con setters
- ✅ Protección de datos sensibles

### Buenas Prácticas

1. **Usa atributos privados** para datos que no deben modificarse directamente
2. **Usa @property** en lugar de getters/setters tradicionales
3. **Valida siempre** los datos en los setters
4. **Propiedades calculadas** para valores que dependen de otros
5. **Solo lectura** para valores que no deben cambiar

### Próximo paso

En el siguiente cuaderno aprenderás:
- **Polimorfismo**: Mismo método, diferentes comportamientos
- Duck typing
- Interfaces implícitas
- Diseño flexible de clases

**¡Ya conoces el tercer pilar de la POO!**